In [2]:
import zipfile

with zipfile.ZipFile('pyvital2.zip') as NRS_vital_zip:
    NRS_vital_zip.extractall()

In [4]:
vital_path = 'PACU_vital/PACU1_1-3'
vital_list = os.listdir(vital_path)
vital_list

['PACU1_3_200717_175105.vital',
 'PACU1_2_200930_000526.vital',
 'PACU1_2_190718_142154.vital',
 'PACU1_1_200224_200759.vital',
 'PACU1_1_200812_095613.vital',
 'PACU1_1_190712_094848.vital',
 'PACU1_1_191024_105656.vital',
 'PACU1_2_200812_103624.vital',
 'PACU1_3_200803_191512.vital',
 'PACU1_2_200317_124925.vital',
 'PACU1_1_190823_094431.vital',
 'PACU1_2_200923_104751.vital',
 'PACU1_1_200316_175815.vital',
 'PACU1_1_190829_193244.vital',
 'PACU1_1_200512_120122.vital',
 'PACU1_1_190430_151645.vital',
 'PACU1_3_200630_134000.vital',
 'PACU1_3_200308_005944.vital',
 'PACU1_3_200811_112456.vital',
 'PACU1_1_200723_151212.vital',
 'PACU1_2_200701_084823.vital',
 'PACU1_3_200821_023317.vital',
 'PACU1_3_190410_023648.vital',
 'PACU1_2_200715_120145.vital',
 'PACU1_1_190523_091823.vital',
 'PACU1_1_201029_165937.vital',
 'PACU1_2_200326_132953.vital',
 'PACU1_3_190820_121227.vital',
 'PACU1_3_201012_151709.vital',
 'PACU1_1_200911_190526.vital',
 'PACU1_3_201005_224405.vital',
 'PACU1_

In [14]:
vals = vitaldb.vital_recs(vital_path+'/PACU1_1_200224_200759.vital', ['SNUADCW/PLETH'], interval = 0.01).flatten()
len(vals)

1

In [10]:
from pyvital2 import pleth_spi

print("created a new group histogram\n")
group_hist_ppga = pleth_spi.Histogram(0, 20, 1000)
group_hist_hbi = pleth_spi.Histogram(240, 2000, 1000)

created a new group histogram



In [6]:
import os, sys
import pickle
import pandas as pd
import numpy as np

#sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('SPI_model'))))
from pyvital2 import arr, vitaldb

vital_path = 'PACU_vital/PACU1_1-3'
vital_list = os.listdir(vital_path)
srate = 100

non_list = []

cnt = 0
# 추가 샘플을 group histo 에 집어넣음
for f_vital in vital_list:
    cnt += 1
    cachepath = "cache/PPG/hbi_ppga_" + f_vital
    
    #이미 저장된 pickle 파일이 있으면 loading -> group histogram을 훈련시킴
    if os.path.exists(cachepath):
        print('({}/{}) '.format(cnt, len(vital_list)), end=' ')
        print('already existing...loading sample... '+f_vital, end='')
        hbi_list, ppga_list = pickle.load(open(cachepath, 'rb'))
        print('done')
        
        #for hbi in hbi_list:
            #group_hist_hbi.learn(hbi)
        #for ppga in ppga_list:
            #group_hist_ppga.learn(ppga)
            
    #저장된 pickle 파일이 없으면 새로 계산해서 저장   
    else:
        print('({}/{}) '.format(cnt, len(vital_list)), end='')
        if not '.vital' in f_vital:
            print('not a vital file')
            non_list.append(f_vital)
            continue
        
        vals = vitaldb.vital_recs(vital_path+'/'+f_vital, ['SNUADCW/PLETH'], interval = 0.01).flatten()
        print('size of', f_vital, ':', len(vals), end='')
        
        if len(vals) == 1:
            print('no data')
            non_list.append(f_vital)
            continue
        
        print('\ndetecting peak...', end='')
        vals = arr.exclude_undefined(vals)
        
        # vals 값이 일정한 경우
        if len(np.unique(vals)) == 1:
            print('no valid data')
            non_list.append(f_vital)
            continue
        
        minlist, maxlist = arr.detect_peaks(vals, 100)
        print('completed...')
        
        hbi_list, ppga_list = [], []
        for j in range(len(maxlist) - 1):
            hbi = (maxlist[j+1] - maxlist[j]) / srate * 1000
            if 240 < hbi < 2000:
                #group_hist_hbi.learn(hbi)
                hbi_list.append(hbi)
            ppga = vals[maxlist[j+1]] - vals[minlist[j]]
            if 0 < ppga < 100:
                #group_hist_ppga.learn(ppga)
                ppga_list.append(ppga)    
                
        pickle.dump((hbi_list, ppga_list), open(cachepath, 'wb'))

(1/2048)  already existing...loading sample... PACU1_3_200717_175105.vitaldone
(2/2048)  already existing...loading sample... PACU1_2_200930_000526.vitaldone
(3/2048)  already existing...loading sample... PACU1_2_190718_142154.vitaldone
(4/2048) size of PACU1_1_200224_200759.vital : 243816
detecting peak...completed...
(5/2048)  already existing...loading sample... PACU1_1_200812_095613.vitaldone
(6/2048)  already existing...loading sample... PACU1_1_190712_094848.vitaldone
(7/2048)  already existing...loading sample... PACU1_1_191024_105656.vitaldone
(8/2048)  already existing...loading sample... PACU1_2_200812_103624.vitaldone
(9/2048)  already existing...loading sample... PACU1_3_200803_191512.vitaldone
(10/2048)  already existing...loading sample... PACU1_2_200317_124925.vitaldone
(11/2048)  already existing...loading sample... PACU1_1_190823_094431.vitaldone
(12/2048)  already existing...loading sample... PACU1_2_200923_104751.vitaldone
(13/2048)  already existing...loading sample

(107/2048) size of PACU1_1_200204_123058.vital : 253130
detecting peak...completed...
(108/2048)  already existing...loading sample... PACU1_1_200522_131543.vitaldone
(109/2048)  already existing...loading sample... PACU1_3_190716_095311.vitaldone
(110/2048)  already existing...loading sample... PACU1_3_200508_183036.vitaldone
(111/2048)  already existing...loading sample... PACU1_3_190816_133726.vitaldone
(112/2048)  already existing...loading sample... PACU1_3_200915_163220.vitaldone
(113/2048)  already existing...loading sample... PACU1_1_191021_094811.vitaldone
(114/2048)  already existing...loading sample... PACU1_2_200602_151101.vitaldone
(115/2048)  already existing...loading sample... PACU1_2_191023_125126.vitaldone
(116/2048)  already existing...loading sample... PACU1_3_200121_110242.vitaldone
(117/2048) size of PACU1_3_200221_224732.vital : 516820
detecting peak...completed...
(118/2048) size of PACU1_3_200214_024720.vital : 241154
detecting peak...completed...
(119/2048)  a

(233/2048) size of PACU1_1_200213_132324.vital : 258642
detecting peak...completed...
(234/2048)  already existing...loading sample... PACU1_2_200716_123803.vitaldone
(235/2048)  already existing...loading sample... PACU1_2_190607_115634.vitaldone
(236/2048)  already existing...loading sample... PACU1_3_190524_131002.vitaldone
(237/2048)  already existing...loading sample... PACU1_1_190704_134024.vitaldone
(238/2048)  already existing...loading sample... PACU1_1_200612_160558.vitaldone
(239/2048)  already existing...loading sample... PACU1_2_200630_224434.vitaldone
(240/2048)  already existing...loading sample... PACU1_2_190820_152951.vitaldone
(241/2048)  already existing...loading sample... PACU1_2_200529_101615.vitaldone
(242/2048)  already existing...loading sample... PACU1_2_201022_091414.vitaldone
(243/2048)  already existing...loading sample... PACU1_3_200818_172716.vitaldone
(244/2048)  already existing...loading sample... PACU1_1_190722_132616.vitaldone
(245/2048)  already exi

(348/2048) size of PACU1_3_200206_224128.vital : 226770
detecting peak...completed...
(349/2048)  already existing...loading sample... PACU1_1_190424_171921.vitaldone
(350/2048)  already existing...loading sample... PACU1_2_190711_085132.vitaldone
(351/2048)  already existing...loading sample... PACU1_1_190715_211026.vitaldone
(352/2048)  already existing...loading sample... PACU1_1_200714_112435.vitaldone
(353/2048)  already existing...loading sample... PACU1_1_200109_162237.vitaldone
(354/2048) size of PACU1_3_200206_163111.vital : 269952
detecting peak...completed...
(355/2048)  already existing...loading sample... PACU1_1_200110_164438.vitaldone
(356/2048) size of PACU1_1_200218_153348.vital : 247930
detecting peak...completed...
(357/2048)  already existing...loading sample... PACU1_1_201007_171720.vitaldone
(358/2048) size of PACU1_2_200221_134644.vital : 240912
detecting peak...completed...
(359/2048)  already existing...loading sample... PACU1_1_200723_125624.vitaldone
(360/204

(481/2048)  already existing...loading sample... PACU1_3_190907_034902.vitaldone
(482/2048)  already existing...loading sample... PACU1_1_201022_090538.vitaldone
(483/2048) size of PACU1_2_200221_160504.vital : 266482
detecting peak...completed...
(484/2048)  already existing...loading sample... PACU1_3_200331_153124.vitaldone
(485/2048)  already existing...loading sample... PACU1_1_200817_154544.vitaldone
(486/2048)  already existing...loading sample... PACU1_1_200107_155217.vitaldone
(487/2048) size of PACU1_2_200211_174603.vital : 87910
detecting peak...completed...
(488/2048) size of PACU1_1_200128_132348.vital : 237128
detecting peak...completed...
(489/2048)  already existing...loading sample... PACU1_1_200528_134815.vitaldone
(490/2048) size of PACU1_3_200129_184824.vital : 270397
detecting peak...completed...
(491/2048)  already existing...loading sample... PACU1_2_200603_153419.vitaldone
(492/2048) size of PACU1_1_200214_212645.vital : 248535
detecting peak...completed...
(493

...

detecting peak...completed...
(1444/2048)  already existing...loading sample... PACU1_1_191231_085337.vitaldone
(1445/2048) size of PACU1_1_200213_140923.vital : 264717
detecting peak...completed...
(1446/2048)  already existing...loading sample... PACU1_3_200109_233312.vitaldone
(1447/2048)  already existing...loading sample... PACU1_1_190603_152550.vitaldone
(1448/2048)  already existing...loading sample... PACU1_1_190411_130403.vitaldone
(1449/2048)  already existing...loading sample... PACU1_1_200723_095535.vitaldone
(1450/2048)  already existing...loading sample... PACU1_2_190612_153210.vitaldone
(1451/2048)  already existing...loading sample... PACU1_2_190717_154349.vitaldone
(1452/2048)  already existing...loading sample... PACU1_1_200513_130640.vitaldone
(1453/2048)  already existing...loading sample... PACU1_3_190904_152424.vitaldone
(1454/2048)  already existing...loading sample... PACU1_3_200924_120358.vitaldone
(1455/2048)  already existing...loading sample... PACU1_3_20100

(1544/2048) size of PACU1_3_200210_212804.vital : 508201
detecting peak...completed...
(1545/2048)  already existing...loading sample... PACU1_3_200708_091543.vitaldone
(1546/2048)  already existing...loading sample... PACU1_2_190828_121020.vitaldone
(1547/2048)  already existing...loading sample... PACU1_3_190708_002936.vitaldone
(1548/2048)  already existing...loading sample... PACU1_3_190820_181030.vitaldone
(1549/2048)  already existing...loading sample... PACU1_3_190503_233552.vitaldone
(1550/2048)  already existing...loading sample... PACU1_3_200911_163759.vitaldone
(1551/2048)  already existing...loading sample... PACU1_2_190828_152507.vitaldone
(1552/2048)  already existing...loading sample... PACU1_3_190822_111857.vitaldone
(1553/2048)  already existing...loading sample... PACU1_2_200317_093413.vitaldone
(1554/2048)  already existing...loading sample... PACU1_1_190719_182807.vitaldone
(1555/2048) size of PACU1_2_200127_094846.vital : 268043
detecting peak...completed...
(1556/

detecting peak...completed...
(1648/2048)  already existing...loading sample... PACU1_3_200302_160235.vitaldone
(1649/2048)  already existing...loading sample... PACU1_3_190819_082733.vitaldone
(1650/2048)  already existing...loading sample... PACU1_3_200303_154819.vitaldone
(1651/2048)  already existing...loading sample... PACU1_2_200309_114930.vitaldone
(1652/2048)  already existing...loading sample... PACU1_2_190703_202359.vitaldone
(1653/2048) size of PACU1_1_200217_111202.vital : 243499
detecting peak...completed...
(1654/2048)  already existing...loading sample... PACU1_2_200609_124225.vitaldone
(1655/2048) size of PACU1_3_200214_175828.vital : 362033
detecting peak...completed...
(1656/2048)  already existing...loading sample... PACU1_2_200309_152150.vitaldone
(1657/2048)  already existing...loading sample... PACU1_3_190902_160501.vitaldone
(1658/2048)  already existing...loading sample... PACU1_1_200714_155006.vitaldone
(1659/2048)  already existing...loading sample... PACU1_3_

(1767/2048) size of PACU1_3_200123_044553.vital : 295362
detecting peak...completed...
(1768/2048) size of PACU1_3_200122_090234.vital : 505660
detecting peak...completed...
(1769/2048)  already existing...loading sample... PACU1_2_190429_121918.vitaldone
(1770/2048)  already existing...loading sample... PACU1_1_200309_134328.vitaldone
(1771/2048)  already existing...loading sample... PACU1_2_200102_175652.vitaldone
(1772/2048)  already existing...loading sample... PACU1_1_190509_144324.vitaldone
(1773/2048)  already existing...loading sample... PACU1_1_190516_203320.vitaldone
(1774/2048)  already existing...loading sample... PACU1_3_190409_114333.vitaldone
(1775/2048)  already existing...loading sample... PACU1_3_200103_145245.vitaldone
(1776/2048)  already existing...loading sample... PACU1_2_200603_133105.vitaldone
(1777/2048)  already existing...loading sample... PACU1_1_190517_140734.vitaldone
(1778/2048)  already existing...loading sample... PACU1_1_190408_202459.vitaldone
(1779/

(1885/2048) size of PACU1_1_200123_104144.vital : 247688
detecting peak...completed...
(1886/2048)  already existing...loading sample... PACU1_1_200615_124257.vitaldone
(1887/2048)  already existing...loading sample... PACU1_1_191226_085511.vitaldone
(1888/2048)  already existing...loading sample... PACU1_1_190607_160350.vitaldone
(1889/2048) size of PACU1_3_200220_233503.vital : 306010
detecting peak...completed...
(1890/2048)  already existing...loading sample... PACU1_1_200928_115238.vitaldone
(1891/2048)  already existing...loading sample... PACU1_3_200103_003134.vitaldone
(1892/2048)  already existing...loading sample... PACU1_1_201008_134520.vitaldone
(1893/2048)  already existing...loading sample... PACU1_3_201024_122437.vitaldone
(1894/2048)  already existing...loading sample... PACU1_2_190705_185939.vitaldone
(1895/2048) size of PACU1_3_200220_172819.vital : 259667
detecting peak...completed...
(1896/2048)  already existing...loading sample... PACU1_1_200925_135316.vitaldone
(

In [5]:
import os, sys
import pickle
import pandas as pd
import numpy as np

#sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('SPI_model'))))
from pyvital2 import arr, vitaldb_mod

vital_path = 'PACU_vital/PACU1_1-3'
f_vital = 'PACU1_1_200224_200759.vital'
vals = vitaldb_mod.vital_recs(vital_path+'/'+f_vital, ['SNUADCW/PLETH'], interval = 0.01).flatten()
print(vals)

[ 27.34375   26.953125  26.953125 ...  13.28125  -10.15625    8.203125]


In [ ]:
vital_path = 'PACU_vital/PACU1_4-6'
vital_list = os.listdir(vital_path)

cnt = -1
# 추가 샘플을 group histo 에 집어넣음
for f_vital in vital_list:
    cnt += 1
    cachepath = "cache/PPG/hbi_ppga_" + f_vital
    
    #이미 저장된 pickle 파일이 있으면 loading -> group histogram을 훈련시킴
    if os.path.exists(cachepath):
        print('loading sample '+f_vital)
        hbi_list, ppga_list = pickle.load(open(cachepath, 'rb'))
        
        for hbi in hbi_list:
            group_hist_ppga.learn(hbi)
        for ppga in ppga_list:
            group_hist_ppga.learn(ppga)
            
    #저장된 pickle 파일이 없으면 새로 계산해서 저장   
    else:
        vals = vitaldb.vital_recs(vital_path+'/'+f_vital, ['SNUADCW/PLETH'], interval = 0.01).flatten()
        if len(vals) == 1:
            print('no data')
            non_list.append(f_vital)
            continue
        
        print('({}/{}) '.format(cnt, len(vital_list)), end='')
        print('size of', f_vital, ':', len(vals))
        print('detecting peak...', end='')
        vals = arr.exclude_undefined(vals)
        minlist, maxlist = arr.detect_peaks(vals, 100)
        print('completed...')
        
        hbi_list, ppga_list = [], []
        for j in range(len(maxlist) - 1):
            hbi = (maxlist[j+1] - maxlist[j]) / srate * 1000
            if 240 < hbi < 2000:
                group_hist_hbi.learn(hbi)
                hbi_list.append(hbi)
            ppga = vals[maxlist[j+1]] - vals[minlist[j]]
            if 0 < ppga < 100:
                group_hist_ppga.learn(ppga)
                ppga_list.append(ppga)    
                
        pickle.dump((hbi_list, ppga_list), open(cachepath, 'wb'))

## SPI test set peak 뽑기
<br>
individual distribution을 구하기 위함

In [17]:
for file_name in vital_list:
    mod_name = file_name.split(',')[1]+','+file_name.split(',')[2]
    os.rename(os.path.join(vital_path,file_name), os.path.join(vital_path,mod_name))

In [19]:
vital_list

['2849,PACU1_4_200108_135548.vital',
 '6875,PACU1_1_200805_103731.vital',
 '7224,PACU1_7_200825_104427.vital',
 '5509,PACU1_2_200529_191241.vital',
 '1258,PACU1_1_190717_084957.vital',
 '4123,PACU1_3_200228_195646.vital',
 '5835,PACU1_9_200612_094447.vital',
 '7245,PACU1_1_200828_144451.vital',
 '2446,PACU1_3_191018_110127.vital',
 '5004,PACU1_8_200508_105237.vital',
 '1391,PACU1_3_190719_154655.vital',
 '7512,PACU1_3_200918_092853.vital',
 '7458,PACU1_1_200916_162304.vital',
 '3998,PACU1_2_200224_104638.vital',
 '4954,PACU1_7_200506_174637.vital',
 '4942,PACU1_11_200504_141412.vital',
 '2537,PACU1_1_191024_125724.vital',
 '2011,PACU1_6_190830_155845.vital',
 '5358,PACU1_8_200526_101944.vital',
 '2436,PACU1_11_191017_140015.vital',
 '2888,PACU1_1_200109_162237.vital',
 '3490,PACU1_10_200205_120438.vital',
 '1640,PACU1_10_190820_163322.vital',
 '8019,PACU1_11_201014_141319.vital',
 '7468,PACU1_4_200917_083845.vital',
 '5364,PACU1_1_200526_123356.vital',
 '6815,PACU1_11_200731_135403.vit

In [10]:
import os, sys
import pickle
import pandas as pd
import numpy as np
#sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('SPI_model'))))
from pyvital2 import arr, pleth_spi, vitaldb

srate = 100
vital_path = 'SPI_test_set'
vital_list = os.listdir(vital_path)

# creating group histogram
print("created a new group histogram\n")
hist_ppga = pleth_spi.Histogram(0, 100, 1000)
hist_hbi = pleth_spi.Histogram(240, 2000, 1000)

non_list, short_ppg_list = [], []
cnt = 0
# 추가 샘플을 group histo 에 집어넣음
for f_vital in vital_list:
    cnt += 1
    cachepath = "cache/SPI_PPG/hbi_ppga_" + f_vital
    
    #이미 저장된 pickle 파일이 있으면 loading -> group histogram을 훈련시킴
    if os.path.exists(cachepath):
        print('already existing peaks of '+f_vital)
            
    #저장된 pickle 파일이 없으면 새로 계산해서 저장   
    else:
        df_vital = pickle.load(open(vital_path+'/'+f_vital,'rb'))
        vals = df_vital['pleth'].tolist()
        #vals = vitaldb.vital_recs(vital_path+'/'+f_vital, ['SNUADCW/PLETH'], interval = 0.01).flatten()
        if len(vals) <= 1:
            print('no data')
            non_list.append(f_vital)
            continue
        if len(vals) < 12000:
            print('ppg data < 2min')
            short_ppg_list.append(f_vital)
            continue
            
        print('({}/{}) '.format(cnt, len(vital_list)), end='')
        print('size of', f_vital, ':', len(vals[:-12000]))
        print('detecting peak...', end='')
        vals = arr.exclude_undefined(vals[:-12000])
        minlist, maxlist = arr.detect_peaks(vals, 100)
        print('completed...')
        
        hbi_list, ppga_list = [], []
        for j in range(len(maxlist) - 1):
            hbi = (maxlist[j+1] - maxlist[j]) / srate * 1000
            if 240 < hbi < 2000:
                hist_hbi.learn(hbi)
                hbi_list.append(hbi)   # 이러면 hbi_list와 ppga_list가 어긋남. spi 계산할땐 문제지만 지금은 단순히 hist 학습
            ppga = vals[maxlist[j+1]] - vals[minlist[j]]
            if 0 < ppga < 100:
                hist_ppga.learn(ppga)
                ppga_list.append(ppga)    
                
        pickle.dump((hbi_list, ppga_list), open(cachepath, 'wb'))
        pickle.dump((hist_hbi, hist_ppga), open('cache/histogram/'+f_vital,'wb'))

created a new group histogram

(1/787) size of 2849,PACU1_4_200108_135548.vital : 223052
detecting peak...completed...
(2/787) size of 6875,PACU1_1_200805_103731.vital : 212779
detecting peak...completed...
(3/787) size of 7224,PACU1_7_200825_104427.vital : 201216
detecting peak...completed...
(4/787) size of 5509,PACU1_2_200529_191241.vital : 91876
detecting peak...completed...
(5/787) size of 1258,PACU1_1_190717_084957.vital : 198355
detecting peak...completed...
(6/787) size of 4123,PACU1_3_200228_195646.vital : 157309
detecting peak...completed...
(7/787) size of 5835,PACU1_9_200612_094447.vital : 79299
detecting peak...completed...
(8/787) size of 7245,PACU1_1_200828_144451.vital : 18839
detecting peak...completed...
(9/787) size of 2446,PACU1_3_191018_110127.vital : 69255
detecting peak...completed...
(10/787) size of 5004,PACU1_8_200508_105237.vital : 32193
detecting peak...completed...
(11/787) size of 1391,PACU1_3_190719_154655.vital : 516335
detecting peak...completed...
ppg 

detecting peak...completed...
(196/787) size of 2236,PACU1_5_190906_164701.vital : 95783
detecting peak...completed...
(197/787) size of 6254,PACU1_5_200702_181420.vital : 81845
detecting peak...completed...
(198/787) size of 7219,PACU1_3_200824_163114.vital : 40520
detecting peak...completed...
(199/787) size of 6959,PACU1_3_200811_112456.vital : 48253
detecting peak...completed...
(200/787) size of 8018,PACU1_6_201014_135931.vital : 140847
detecting peak...completed...
(201/787) size of 4411,PACU1_9_200310_165006.vital : 47264
detecting peak...completed...
(202/787) size of 5792,PACU1_1_200610_202606.vital : 11284
detecting peak...completed...
(203/787) size of 3313,PACU1_9_200129_100629.vital : 38939
detecting peak...completed...
(204/787) size of 4381,PACU1_7_200309_165443.vital : 49669
detecting peak...completed...
(205/787) size of 690,PACU1_5_190529_101102.vital : 101792
detecting peak...completed...
(206/787) size of 5316,PACU1_8_200522_110921.vital : 81844
detecting peak...com

detecting peak...completed...
(293/787) size of 1809,PACU1_4_190822_203857.vital : 174133
detecting peak...completed...
(294/787) size of 3244,PACU1_12_200122_163312.vital : 88679
detecting peak...completed...
(295/787) size of 5148,PACU1_1_200514_153414.vital : 142355
detecting peak...completed...
(296/787) size of 7593,PACU1_1_200922_150810.vital : 208874
detecting peak...completed...
(297/787) size of 4251,PACU1_5_200304_124721.vital : 33876
detecting peak...completed...
(298/787) size of 3124,PACU1_3_200117_165846.vital : 115128
detecting peak...completed...
(299/787) size of 2318,PACU1_8_190910_165227.vital : 33204
detecting peak...completed...
ppg data < 2min
(301/787) size of 2001,PACU1_10_190830_135223.vital : 153533
detecting peak...completed...
(302/787) size of 7396,PACU1_6_200915_112132.vital : 38768
detecting peak...completed...
(303/787) size of 3822,PACU1_2_200218_185629.vital : 129052
detecting peak...completed...
(304/787) size of 7012,PACU1_2_200812_164026.vital : 753

detecting peak...completed...
(391/787) size of 4049,PACU1_9_200225_172708.vital : 65174
detecting peak...completed...
(392/787) size of 2479,PACU1_3_191021_093930.vital : 321014
detecting peak...completed...
(393/787) size of 3233,PACU1_1_200122_134827.vital : 57262
detecting peak...completed...
(394/787) size of 2787,PACU1_8_200106_163626.vital : 129310
detecting peak...completed...
(395/787) size of 5487,PACU1_6_200529_121341.vital : 37890
detecting peak...completed...
(396/787) size of 5790,PACU1_1_200610_191621.vital : 9800
detecting peak...completed...
(397/787) size of 7373,PACU1_3_200911_195111.vital : 100831
detecting peak...completed...
(398/787) size of 157,PACU1_4_190430_011154.vital : 126458
detecting peak...completed...
(399/787) size of 5973,PACU1_10_200617_142506.vital : 17300
detecting peak...completed...
(400/787) size of 7320,PACU1_3_200910_131322.vital : 117705
detecting peak...completed...
(401/787) size of 6091,PACU1_7_200624_093158.vital : 156169
detecting peak..

detecting peak...completed...
(584/787) size of 1074,PACU1_10_190709_172441.vital : 19877
detecting peak...completed...
(585/787) size of 5366,PACU1_11_200526_131650.vital : 96810
detecting peak...completed...
(586/787) size of 3885,PACU1_4_200219_210246.vital : 1247
detecting peak...completed...
(587/787) size of 3813,PACU1_4_200218_154434.vital : 230539
detecting peak...completed...
(588/787) size of 5497,PACU1_10_200529_151443.vital : 49686
detecting peak...completed...
ppg data < 2min
(590/787) size of 7194,PACU1_1_200821_165245.vital : 61387
detecting peak...completed...
(591/787) size of 6073,PACU1_4_200623_151318.vital : 88156
detecting peak...completed...
(592/787) size of 474,PACU1_7_190515_162217.vital : 4193
detecting peak...completed...
(593/787) size of 5147,PACU1_1_200514_153414.vital : 22355
detecting peak...completed...
ppg data < 2min
(595/787) size of 5991,PACU1_6_200618_083629.vital : 128976
detecting peak...completed...
(596/787) size of 3987,PACU1_3_200221_224732.v

detecting peak...completed...
(684/787) size of 8229,PACU1_11_201026_132629.vital : 39050
detecting peak...completed...
(685/787) size of 302,PACU1_12_190507_153946.vital : 79258
detecting peak...completed...
(686/787) size of 1734,PACU1_10_190821_154935.vital : 20498
detecting peak...completed...
(687/787) size of 390,PACU1_2_190513_214102.vital : 11727
detecting peak...completed...
(688/787) size of 7143,PACU1_10_200820_103206.vital : 95338
detecting peak...completed...
(689/787) size of 3894,PACU1_1_200220_102905.vital : 83402
detecting peak...completed...
(690/787) size of 2386,PACU1_12_190917_105505.vital : 119414
detecting peak...completed...
(691/787) size of 7537,PACU1_2_200918_163529.vital : 134991
detecting peak...completed...
ppg data < 2min
(693/787) size of 1345,PACU1_5_190718_182050.vital : 102968
detecting peak...completed...
(694/787) size of 4733,PACU1_4_200323_140030.vital : 74925
detecting peak...completed...
(695/787) size of 1340,PACU1_1_190718_165707.vital : 15520

detecting peak...completed...
(779/787) size of 7116,PACU1_2_200819_152037.vital : 74124
detecting peak...completed...
(780/787) size of 1870,PACU1_10_190827_134744.vital : 31522
detecting peak...completed...
(781/787) size of 1760,PACU1_5_190821_201717.vital : 64196
detecting peak...completed...
(782/787) size of 236,PACU1_4_190503_094652.vital : 36665
detecting peak...completed...
(783/787) size of 2347,PACU1_8_190911_135025.vital : 45334
detecting peak...completed...
(784/787) size of 307,PACU1_4_190507_171358.vital : 144124
detecting peak...completed...
(785/787) size of 3489,PACU1_10_200205_120438.vital : 50106
detecting peak...completed...
(786/787) size of 7086,PACU1_7_200818_145515.vital : 196427
detecting peak...completed...
(787/787) size of 4964,PACU1_9_200507_091727.vital : 93148
detecting peak...completed...


In [11]:
short_ppg_list

['7512,PACU1_3_200918_092853.vital',
 '5409,PACU1_4_200527_135854.vital',
 '301,PACU1_12_190507_153946.vital',
 '1689,PACU1_3_190821_084813.vital',
 '7136,PACU1_1_200820_093916.vital',
 '4129,PACU1_3_200302_100836.vital',
 '176,PACU1_12_190430_151349.vital',
 '7851,PACU1_7_201007_111324.vital',
 '295,PACU1_11_190507_112917.vital',
 '6974,PACU1_8_200811_162825.vital',
 '1456,PACU1_5_190723_165846.vital',
 '8270,PACU1_2_201027_153435.vital',
 '803,PACU1_12_190607_141912.vital',
 '6966,PACU1_2_200811_124816.vital',
 '2045,PACU1_4_190902_234827.vital',
 '4711,PACU1_3_200320_163212.vital',
 '2831,PACU1_2_200108_092327.vital',
 '3768,PACU1_1_200217_115421.vital',
 '5936,PACU1_12_200616_152932.vital',
 '1703,PACU1_10_190821_102951.vital',
 '4631,PACU1_1_200318_161331.vital',
 '1737,PACU1_3_190821_160843.vital',
 '2123,PACU1_5_190904_154406.vital',
 '5982,PACU1_4_200617_163518.vital',
 '970,PACU1_5_190704_190828.vital',
 '3731,PACU1_10_200214_134908.vital',
 '86,PACU1_1_190419_195900.vital',
 

In [4]:
import os, sys
import pickle
import pandas as pd
import numpy as np
#sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('SPI_model'))))
from pyvital2 import arr, pleth_spi, vitaldb

srate = 100
vital_path = 'preop_vital/preop'
vital_list = os.listdir(vital_path)


non_list, short_ppg_list = [], []
cnt = 0
# 추가 샘플을 group histo 에 집어넣음
for f_vital in vital_list:
    cnt += 1
    cachepath = "cache/PPG_preop/hbi_ppga_" + f_vital
    
    #이미 저장된 pickle 파일이 있으면 loading -> group histogram을 훈련시킴
    if os.path.exists(cachepath):
        print('already existing peaks of '+f_vital)
            
    #저장된 pickle 파일이 없으면 새로 계산해서 저장   
    else:
        #df_vital = pickle.load(open(vital_path+'/'+f_vital,'rb'))
        #vals = df_vital['Pleth'].tolist()
        vals = vitaldb.vital_recs(vital_path+'/'+f_vital, ['SNUADCW/PLETH'], interval = 0.01).flatten()
        if len(vals) <= 1:
            print('no data')
            non_list.append(f_vital)
            continue
            
        print('({}/{}) '.format(cnt, len(vital_list)), end='')
        print('size of', f_vital, ':', len(vals))
        print('detecting peak...', end='')
        vals = arr.exclude_undefined(vals)
        minlist, maxlist = arr.detect_peaks(vals, 100)
        print('completed...')
        
        hbi_list, ppga_list = [], []
        for j in range(len(maxlist) - 1):
            hbi = (maxlist[j+1] - maxlist[j]) / srate * 1000
            if 240 < hbi < 2000:
                hbi_list.append(hbi)   # 이러면 hbi_list와 ppga_list가 어긋남. spi 계산할땐 문제지만 지금은 단순히 hist 학습
            ppga = vals[maxlist[j+1]] - vals[minlist[j]]
            if 0 < ppga < 100:
                ppga_list.append(ppga)    
                
        pickle.dump((hbi_list, ppga_list), open(cachepath, 'wb'))
        #pickle.dump((hist_hbi, hist_ppga), open('cache/histogram/'+f_vital,'wb'))

OSError: Not a gzipped file (b'\x80\x04')